In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

    
    
    
def Belief(State,p,r):
   
    equilibrium = r/(1-p+r);
    
    if State[0]== 0 and State[1]> 0:
            Belief = (r-r*(p-r)**(State[1]))/(1+r-p);
    elif State[0]== 1 and State[1]> 0:
            Belief = (r+(1-p)*(p-r)**(State[1]))/(1+r-p);
            
    else:
        
        Belief = equilibrium;

    return Belief;
            
        
def WhittleIndex(State,p,r):
    WhittleIndex = 0.;

    
    if State[0]== 0 and State[1]>0:
        l= State[1];
        x= Belief([0,l],p,r);
        y=Belief([0,l+1],p,r);
        WhittleIndex =((x-y)*(l+1)+y)/(1-p+(x-y)*l+y); 
    elif State[0] == 1:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
    
    else:
        WhittleIndex = r/((1-p)*(1+r-p)+r);
            
    return WhittleIndex;

def update(S, C,Reward,alpha,p,r,N):
        #the update process of each timestep.
        
        #S is the list of states of all channels.
        W = [0.]*N;
        Y = [0.]*N;
        Action = [0]*N;
        Threshold = 0.;
        Seed = np.random.rand(N);
        
       #Get the Threshold. 
        for i in range(N):
            Y[i]= WhittleIndex(S[i],p,r);
        
        Y.sort();
        Threshold = Y[int((1-alpha)*N)];
        
        
        
       #Get the Action vector.
        for i in range(N):
            W[i]= WhittleIndex(S[i],p,r);
    
        h=0;
        for i in range(N):
            if W[i]>= Threshold:
                Action[i]=1;
                h=h+1;
            if h >= alpha*N:
                break;
            
       #The update process.
       
        for i in range(N):
        
            if Action[i] == 0 and S[i][1]>0:
                S[i][1]= S[i][1]+1;
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and S[i][1]==0:
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    S[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<p)==True);
                    
                else:
                    S[i]= [0,1];
                    C[i]= int((Seed[i]<r)==True);
        
        
        return Reward;
    


In [8]:
print(update([[0,0],[0,0],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,0],0,0.4,0.6,0.3,10));

1


In [9]:
def onetrial(S,C,Reward,timesteps,alpha,p,r,N):

    for i in range(timesteps):
        Reward = update(S,C,Reward,alpha,p,r,N);
    return Reward;

In [10]:
print(onetrial([[0,0],[0,0],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,1],0,100,0.4,0.6,0.2,10));

178


In [11]:
def mc(S,Reward,ntrials,timesteps,alpha,p,r,N):
    equilibrium = r/(1-p+r);
    
   
    bigS = [S]*(ntrials);
    for i in range(ntrials):
        C = [0]*N;
        Seed = np.random.rand(N);
        for j in range(N):
            C[j]= int((Seed[j]< equilibrium) == True);
    
        Reward = onetrial(bigS[i],C,Reward,timesteps,alpha,p,r,N);
        
    lowerbound = 100*Reward/(timesteps*ntrials*N);
    
    print(lowerbound);


In [12]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.6,0.3,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#The upperbound is 20.613

20.21306
20.19057
20.215253333333333
20.233785
20.235248
20.291214
20.334375


In [13]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.7,0.4,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#The upperbound is 26.213

25.9884
25.96761
25.963073333333334
25.965595
25.986276
26.009266
26.052708


In [14]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.7,0.1,N);
    return lb;

y= [lb(10),lb(20),lb(30),lb(40),lb(50),lb(100),lb(200)];

#The upperbound is 16.360

15.06688
15.0089
15.08128
15.192605
15.241972
15.452184
15.594539


In [4]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.8,0.2,N);
    return lb;

y= [lb(5),lb(10),lb(15),lb(20),lb(25),lb(30),lb(35),lb(40),lb(45),lb(50),lb(100),lb(200)];

#The upperbound is 28.191

26.34472
27.0817
27.203146666666665
27.30157
27.364488
27.39488
27.446142857142856
27.46561
27.476506666666666
27.531092
27.600314
27.700184


In [4]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.85,0.25,N);
    return lb;

y= [lb(5),lb(10),lb(15),lb(20),lb(25),lb(30),lb(35),lb(40),lb(45),lb(50),lb(100),lb(200)];


#The upperbound is 32.181

31.11776
31.66474
31.74108
31.81541
31.829632
31.84942
31.852005714285713
31.876635
31.859368888888888
31.878928
31.930518
31.961919


In [4]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.8,0.55,N);
    return lb;

y= [lb(5),lb(10),lb(15),lb(20),lb(25),lb(30),lb(35),lb(40),lb(45),lb(50),lb(100),lb(200)];

#The upperbound is 31.428

31.31328
31.33778
31.35488
31.34725
31.328696
31.337653333333332
31.33724
31.34773
31.335395555555557
31.321436
31.349174
31.356404


In [4]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.9,0.6,N);
    return lb;

y= [lb(5),lb(10),lb(15),lb(20),lb(25),lb(30),lb(35),lb(40),lb(45),lb(50),lb(100),lb(200)];


#The upperbound is 35.821

35.81112
35.80998
35.81238666666667
35.79548
35.793384
35.796726666666665
35.801634285714286
35.790925
35.78768888888889
35.786096
35.79165
35.790741


In [4]:
def lb(N):
    lb= mc([[0,0]]*N,0,500,1000,0.4,0.4,0.2,N);
    return lb;

y= [lb(5),lb(10),lb(15),lb(20),lb(25),lb(30),lb(35),lb(40),lb(45),lb(50),lb(100),lb(200)];


#The upperbound is 11.704

11.27528
11.31232
11.341306666666666
11.34234
11.357856
11.387486666666666
11.39444
11.39734
11.39568
11.429376
11.475024
11.537278
